In [1]:
# Load the TensorBoard notebook extension
# In colab this can be done prior to the pip install above.
# I added the pip install since it was done in the dev summit 2019 video
%load_ext tensorboard.notebook

In [3]:
# google collab 

#from google.colab import drive
#drive.mount('/content/gdrive')

In [4]:
# clear any logs from previous runs
!rm -rf ./logs

In [5]:
import datetime
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import os


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [6]:
print(tf.__version__)

2.0.0-alpha0


In [7]:
#from google.colab import files
#uploaded = files.upload()

In [17]:
# google collab version
#CSV_FILE_NAME = "usgs_gsvb_v2.csv"
# jupyter notebook version
CSV_FILE_NAME = "../csv/usgs_gsvb_v2.csv"
os.environ['CSV_FILE_NAME'] = CSV_FILE_NAME

In [18]:
%%bash
head ${CSV_FILE_NAME}

usgs,site_no,date,tidal_high,tidal_low,precipitation_inches,lat,long
GSVB,33670001,2019-03-08,-6.7913385826771648,-6.7257217847769031,,36.735303,-76.011602
GSVB,33670001,2019-03-19,-6.8569553805774275,-6.758530183727034,,36.735303,-76.011602
GSVB,33670001,2019-02-21,1.9356955380577427,2.1325459317585302,,36.735303,-76.011602
GSVB,33670001,2019-03-11,-6.66010498687664,-6.4304461942257216,,36.735303,-76.011602
GSVB,33670001,2019-04-04,1.9028871391076114,2.0669291338582676,,36.735303,-76.011602
GSVB,33670001,2019-02-14,1.7716535433070866,1.9356955380577427,,36.735303,-76.011602
GSVB,33670001,2019-04-14,2.2309711286089238,2.39501312335958,,36.735303,-76.011602
GSVB,33670001,2019-02-24,2.65748031496063,2.9527559055118111,,36.735303,-76.011602
GSVB,33670001,2019-01-12,2.2965879265091864,2.39501312335958,,36.735303,-76.011602


In [20]:
%%bash
# this csv file has 9875 lines corresponding to records (plus one header row)
wc -l ${CSV_FILE_NAME}

    9876 ../csv/usgs_gsvb_v2.csv


In [22]:
# for debug purposes, this allows lines to be skipped during read of csv file
num_lines_to_skip = 0

In [27]:
# since some of the fields have missing values, use the optional
# capability of the api
text_lines_ds = tf.data.experimental.CsvDataset(filenames=CSV_FILE_NAME,
                                                record_defaults = [tf.string,                              # Req field: usgs              - STRING  
                                                                   tf.int64,                               # Req field: site_no           - INTEGER  
                                                                   tf.string,                              # Req field: date              - DATE -> STRING  
                                                                   tf.constant([1.2], dtype=tf.float32),   # Opt field: default avg 1.2 Tidal High (feet) - FLOAT
                                                                   tf.constant([-0.89], dtype=tf.float32),   # Opt field: Tidal Low (feet)  - FLOAT
                                                                   tf.constant([0.117], dtype=tf.float32), # Opt field, default annual/365: Precipitation (inches) FLOAT  
                                                                   tf.float32,                             # Req Lat FLOAT
                                                                   tf.float32                              # Long FLOAT
                                                ],  
                                                header=True
                                               ).skip(num_lines_to_skip)

In [28]:
# Print the initial specified lines in the dataset
# This shows how the missing data is overridden with the defaults for precipitation.
# Also shows how when data is present, eg tidal high it is not overwritten
def my_dataset_head(a_ds, limit=5):
    for i in a_ds:
        limit = limit - 1
        if (limit < 0):
            break
        tf.print(i)
        
my_dataset_head(text_lines_ds, limit=10)

("GSVB", 33670001, "2019-03-08", -6.79133844, -6.72572184, 0.117, 36.735302, -76.0116043)
("GSVB", 33670001, "2019-03-19", -6.85695553, -6.75853, 0.117, 36.735302, -76.0116043)
("GSVB", 33670001, "2019-02-21", 1.93569553, 2.13254595, 0.117, 36.735302, -76.0116043)
("GSVB", 33670001, "2019-03-11", -6.66010475, -6.43044615, 0.117, 36.735302, -76.0116043)
("GSVB", 33670001, "2019-04-04", 1.90288711, 2.0669291, 0.117, 36.735302, -76.0116043)
("GSVB", 33670001, "2019-02-14", 1.77165353, 1.93569553, 0.117, 36.735302, -76.0116043)
("GSVB", 33670001, "2019-04-14", 2.2309711, 2.39501309, 0.117, 36.735302, -76.0116043)
("GSVB", 33670001, "2019-02-24", 2.65748024, 2.95275593, 0.117, 36.735302, -76.0116043)
("GSVB", 33670001, "2019-01-12", 2.29658794, 2.39501309, 0.117, 36.735302, -76.0116043)
("GSVB", 33670001, "2019-02-07", 2.0341208, 2.19816279, 0.117, 36.735302, -76.0116043)


In [29]:
# I don't know how to determine the number or records with TF like pandas.shape
# So iterate through all records and find the number of records.
# Added bonus, it showed that other fields were missing data so the 
# cvsdataset read above needed to be adjusted.
def determine_size_of_dataset(a_ds):
    num_records = 0
    for i in a_ds:
        num_records = num_records + 1
    return(num_records)
 
# BigQuery says that we have 9875 records
DATASET_SIZE = determine_size_of_dataset(text_lines_ds)
print("num records in dataset is {}".format(DATASET_SIZE))

num records in dataset is 9875


In [30]:
# my_ds is a copy of the text_lines_ds so we can manipulate it without rereading the cvs file
my_ds = text_lines_ds

In [31]:
# experiment with shard()
# create two shards and see how the data is split between them.
shard_0_ds = my_ds.shard(2,0)
shard_1_ds = my_ds.shard(2,1)

In [32]:
print("num records in shard 0 dataset is {}".format(determine_size_of_dataset(shard_0_ds)))
print("num records in shard 1 dataset is {}".format(determine_size_of_dataset(shard_1_ds)))

num records in shard 0 dataset is 4938
num records in shard 1 dataset is 4937


In [33]:
# dump the head of shard 0
my_dataset_head(shard_0_ds)

("GSVB", 33670001, "2019-03-08", -6.79133844, -6.72572184, 0.117, 36.735302, -76.0116043)
("GSVB", 33670001, "2019-02-21", 1.93569553, 2.13254595, 0.117, 36.735302, -76.0116043)
("GSVB", 33670001, "2019-04-04", 1.90288711, 2.0669291, 0.117, 36.735302, -76.0116043)
("GSVB", 33670001, "2019-04-14", 2.2309711, 2.39501309, 0.117, 36.735302, -76.0116043)
("GSVB", 33670001, "2019-01-12", 2.29658794, 2.39501309, 0.117, 36.735302, -76.0116043)


In [34]:
# dump the head of shard 1
my_dataset_head(shard_1_ds)

("GSVB", 33670001, "2019-03-19", -6.85695553, -6.75853, 0.117, 36.735302, -76.0116043)
("GSVB", 33670001, "2019-03-11", -6.66010475, -6.43044615, 0.117, 36.735302, -76.0116043)
("GSVB", 33670001, "2019-02-14", 1.77165353, 1.93569553, 0.117, 36.735302, -76.0116043)
("GSVB", 33670001, "2019-02-24", 2.65748024, 2.95275593, 0.117, 36.735302, -76.0116043)
("GSVB", 33670001, "2019-02-07", 2.0341208, 2.19816279, 0.117, 36.735302, -76.0116043)


In [35]:
# the dumps shows that the dataset is split by picking every other line since we split on two shards.

# Split dataset (or dataframe) into three subsets, train, test and valid

In [36]:
# From a stackoverflow post on how to split train, test, and validation 
# datasets using the TF primatives.
# https://stackoverflow.com/questions/48213766/split-a-dataset-created-by-tensorflow-dataset-api-in-to-train-and-test

In [37]:
# Goal is to have three datasets
# Training is 70%
# Validation is 15%
# Testing is 15%

train_size = int(0.7 * DATASET_SIZE)
test_size = int(0.15 * DATASET_SIZE)
validation_size = int(0.15 * DATASET_SIZE)

In [38]:
# shuffle the entire dataset
full_ds = my_ds.shuffle(2*DATASET_SIZE)
print("num records in full shuffled dataset is {}".format(determine_size_of_dataset(full_ds)))

num records in full shuffled dataset is 9875


In [39]:
# Good. The dataset did not change in size after shuffle.  Is it reordered?
my_dataset_head(full_ds)

("USGS", 2043155, "2016-10-12", 2.05, 0.97, 0.117, 36.7883606, -76.0707779)
("GSVB", 33670002, "2019-01-05", 0.623359561, 3.44488192, 0.117, 36.9035187, -76.0957)
("USGS", 204288539, "2017-12-13", 1.44, -1.98, 0.117, 36.7116928, -76.268219)
("USGS", 204291317, "2016-07-31", 1.7, -1.14, 1.9, 36.8433037, -76.1244736)
("USGS", 204291317, "2017-12-15", 1.11, -1.55, 0, 36.8433037, -76.1244736)


In [40]:
# It appears so.  
# Lets create the three datasets using take() and skip() as shown in the post.
train_ds = full_ds.take(train_size)  # take from start to train_size for the training set.
test_ds = full_ds.skip(train_size).take(test_size) # skip past the training and take the test_size for test set.
valid_ds = full_ds.skip(train_size).skip(test_size).take(validation_size) # likewise skip twice and take for valid set

In [41]:
# Lets see if sizes are correct
print("num records in full train dataset is {}".format(determine_size_of_dataset(train_ds)))
print("num records in full test dataset is {}".format(determine_size_of_dataset(test_ds)))
print("num records in full valid dataset is {}".format(determine_size_of_dataset(valid_ds)))

num records in full train dataset is 6912
num records in full test dataset is 1481
num records in full valid dataset is 1481


In [61]:
# Lets see if data looks valid on first and last entry of each dataset
def my_dataset_first_and_last(a_ds, size=1):    
    count = 0
    for i in a_ds:
        if (count == 0) or (count == size - 1):
            tf.print(i)
        count = count + 1

# alternative method        
# does not work with tf2.0
#def my_dataset_first_and_last2(a_ds, size=1): 
#    iterator = tf.data.Iterator.from_structure(a_ds.output_types,
#                                               a_ds.output_shapes)
#    next_element = iterator.get_next()
#    tf.print(next_element)

    
# alternative method        
# does not work
#def my_dataset_first_and_last2(a_ds, size=1): 
#    tf.print(a_ds[0])
    
    

In [62]:
# dump first and last of train_ds
my_dataset_first_and_last(train_ds, train_size)
#print("alternative method")
#my_dataset_first_and_last2(train_ds, train_size)

("USGS", 2043155, "2016-10-12", 2.05, 0.97, 0.117, 36.7883606, -76.0707779)
("USGS", 204295505, "2017-03-31", 1.46, -0.96, 1.31, 36.8592796, -75.9844742)


In [63]:
# dump first and last of test_ds
my_dataset_first_and_last(test_ds, test_size)

("GSVB", 33670004, "2019-04-01", -0.426509172, 0.590551198, 0.117, 36.6927, -75.9263763)
("USGS", 204297575, "2017-04-01", 2.79, -1.96, 0, 36.8247795, -75.9852524)


In [64]:
# dump first and last of valid_ds
my_dataset_first_and_last(valid_ds, validation_size)

("USGS", 2043155, "2017-07-05", 1.4, -0.58, 0.117, 36.7883606, -76.0707779)
("USGS", 204288539, "2017-10-01", 2.23, -0.19, 0.117, 36.7116928, -76.268219)


In [68]:
# that simple spot check looks good, but it worries me that none of these samples
# show a GSVB sample.  I'll assume its valid.

# Look at the dataset

In [70]:
sns.pairplot(train_ds[["usgs","site_no","date","tidal_high","tidal_low","precipitation_inches","lat","long"]], diag_kind="kde")

TypeError: 'TakeDataset' object is not subscriptable

In [73]:
train_stats = train_ds.describe()

AttributeError: 'TakeDataset' object has no attribute 'describe'

In [74]:
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(train_ds)
normed_test_data = norm(test_ds)

NameError: name 'train_stats' is not defined

In [75]:
train_ds

<TakeDataset shapes: ((), (), (), (), (), (), (), ()), types: (tf.string, tf.int64, tf.string, tf.float32, tf.float32, tf.float32, tf.float32, tf.float32)>

In [89]:
tf.print(train_ds.take(3))

<TakeDataset shapes: ((), (), (), (), (), (), (), ()), types: (tf.string, tf.int64, tf.string, tf.float32, tf.float32, tf.float32, tf.float32, tf.float32)>


In [78]:
def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation=tf.nn.relu, input_shape=[len(train_ds.keys())]),
    layers.Dense(64, activation=tf.nn.relu),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mean_squared_error',
                optimizer=optimizer,
                metrics=['mean_absolute_error', 'mean_squared_error'])
  return model

In [79]:
model = build_model()

AttributeError: 'TakeDataset' object has no attribute 'keys'

In [96]:
# another method
(x_train, y_train),(x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0





NameError: name 'fashion_mnist' is not defined

In [106]:
# https://www.tensorflow.org/alpha/guide/keras/overview#train_from_tfdata_datasets

In [105]:
foo_ds = tf.data.Dataset.from_tensor_slices((train_ds, labels))
foo_ds = foo_ds.batch(32)



NameError: name 'labels' is not defined

In [97]:
def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

In [104]:
def train_model():
  
    model = create_model()
    model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

    logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    
    tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

#  model.fit(x=x_train, 
#            y=y_train, 
#            epochs=5, 
#            validation_data=(x_test, y_test), 
#            callbacks=[tensorboard_callback])#

    
#    model.fit(train_ds, 
#             epochs=5, 
#             validation_data=(test_ds), 
#             callbacks=[tensorboard_callback])
  
    model.fit(foo_ds, epochs=10, steps_per_epoch=30)
    
train_model()

NameError: name 'foo_ds' is not defined

In [111]:
import itertools

def gen():
  for i in itertools.count(1):
    yield (i, [1] * i)

ds = tf.data.Dataset.from_generator(
    gen, 
    (tf.int64, tf.int64), 
    (tf.TensorShape([]), 
     tf.TensorShape([None])))

for value in ds.take(2):
  print (value)
# (1, array([1]))
# (2, array([1, 1]))

(<tf.Tensor: id=919064, shape=(), dtype=int64, numpy=1>, <tf.Tensor: id=919065, shape=(1,), dtype=int64, numpy=array([1])>)
(<tf.Tensor: id=919068, shape=(), dtype=int64, numpy=2>, <tf.Tensor: id=919069, shape=(2,), dtype=int64, numpy=array([1, 1])>)


In [123]:
# 0     1        2     3           4          5                     6    7
# usgs, site_no, date, tidal_high, tidal_low, precipitation_inches, lat, long
for i in train_ds:
    # can not subset by column names. ie. i['usgs'] does not work
    # can subset by column index. first column is 0 and inclusive
    #                             last column is exclusive
    #                             e.g. [0:8] picks cols 0-7
    #tf.print(i[0:8])
    #
    # print label and then features
    tf.print([i[3], i[2], i[5], i[6], i[7]])
    

[2.05, "2016-10-12", 0.117, 36.7883606, -76.0707779]
[0.623359561, "2019-01-05", 0.117, 36.9035187, -76.0957]
[1.44, "2017-12-13", 0.117, 36.7116928, -76.268219]
[1.7, "2016-07-31", 1.9, 36.8433037, -76.1244736]
[1.11, "2017-12-15", 0, 36.8433037, -76.1244736]
[2.1, "2017-05-05", 0.117, 36.7883606, -76.0707779]
[0.59, "2016-11-24", 0.11, 36.8433037, -76.1244736]
[1.32, "2017-10-06", 0, 36.8433037, -76.1244736]
[1.49, "2017-05-01", 0.117, 36.7116928, -76.268219]
[2.07, "2016-11-04", 0.68, 36.8592796, -75.9844742]
[2.52, "2018-11-24", 0.117, 36.7116928, -76.268219]
[2.51, "2018-03-29", 0, 36.8247795, -75.9852524]
[1.09, "2018-02-04", 0.32, 36.8272209, -76.1655579]
[-0.688976407, "2019-01-23", 0.117, 36.7211685, -76.0335922]
[1.87, "2017-07-05", 0.117, 36.7116928, -76.268219]
[-0.393700778, "2019-02-09", 0.117, 36.852, -76.2885666]
[1.2, "2017-06-03", 0.7, 36.6798058, -75.984]
[1.85, "2018-11-29", 0, 36.8247795, -75.9852524]
[-0.590551198, "2019-02-12", 0.117, 36.852, -76.2885666]
[0.9186

[1.96850395, "2019-04-13", 0.117, 36.735302, -76.0116043]
[1.23, "2018-12-31", 0.08, 36.8592796, -75.9844742]
[2.65, "2016-12-06", 0.8, 36.8272209, -76.1655579]
[1.6, "2016-07-23", 0, 36.8272209, -76.1655579]
[1.25, "2017-10-30", 0, 36.8411407, -76.0571365]
[1.61, "2017-07-21", 0, 36.8433037, -76.1244736]
[0.98, "2017-02-08", 0.01, 36.8411407, -76.0571365]
[1.09, "2016-07-04", 0.08, 36.8592796, -75.9844742]
[5.27, "2016-10-09", 3.84, 36.8411407, -76.0571365]
[1.55, "2017-07-02", 0, 36.8247795, -75.9852524]
[-0.262467206, "2019-03-09", 0.117, 36.7094803, -75.9884]
[1.18, "2016-12-10", 0, 36.8272209, -76.1655579]
[0.69, "2019-02-01", 0, 36.8272209, -76.1655579]
[1.2, "2016-07-29", 0.6, 36.6798058, -75.984]
[1.91, "2018-08-25", 0.117, 36.7116928, -76.268219]
[2.96, "2016-06-02", 0, 36.8247795, -75.9852524]
[3.31, "2017-05-27", 0.02, 36.8247795, -75.9852524]
[-0.131233603, "2019-01-02", 0.117, 36.6927, -75.9263763]
[-0.853018343, "2019-02-12", 0.117, 36.7316933, -75.9847717]
[0.623359561, 

[1.68, "2017-07-16", 0.117, 36.7883606, -76.0707779]
[1.29, "2018-06-30", 0.117, 36.7883606, -76.0707779]
[2.25, "2016-05-11", 0.19, 36.8247795, -75.9852524]
[1.03, "2016-10-23", 0, 36.8411407, -76.0571365]
[0, "2018-10-22", 0.117, 36.8157768, -76.0609131]
[1.24671912, "2019-03-16", 0.117, 36.852, -76.2885666]
[1.1, "2016-12-30", 0, 36.8433037, -76.1244736]
[1.2, "2017-05-31", 1.03, 36.6798058, -75.984]
[0.36, "2019-02-15", 0, 36.8592796, -75.9844742]
[1.85, "2017-04-23", 0.28, 36.8592796, -75.9844742]
[2.15, "2018-05-30", 0.117, 36.8272209, -76.1655579]
[0, "2018-08-16", 0.117, 36.8157768, -76.0609131]
[1.15, "2017-02-14", 0.117, 36.7116928, -76.268219]
[2.6, "2018-09-24", 0.01, 36.8433037, -76.1244736]
[1.27, "2017-10-27", 0, 36.8272209, -76.1655579]
[1.67, "2018-01-05", 0, 36.8272209, -76.1655579]
[1.67, "2018-02-17", 0.117, 36.7116928, -76.268219]
[1.9, "2018-12-03", 0.117, 36.7883606, -76.0707779]
[1.26, "2017-06-04", 0.117, 36.7883606, -76.0707779]
[1.18, "2017-01-20", 0.117, 36.

[0, "2017-11-28", -0.05, 36.6798058, -75.984]
[1.2, "2019-03-15", 0.117, 36.8157768, -76.0609131]
[0.82, "2017-07-13", 0, 36.8592796, -75.9844742]
[0.82, "2019-03-13", 0, 36.8592796, -75.9844742]
[0.85, "2019-01-12", 0.06, 36.8592796, -75.9844742]
[1.84, "2019-01-17", 0, 36.8272209, -76.1655579]
[2.17, "2017-07-26", 0.117, 36.7116928, -76.268219]
[1.99, "2017-07-09", 0.02, 36.8247795, -75.9852524]
[1.09, "2017-04-16", 0.117, 36.7116928, -76.268219]
[0.95, "2018-10-17", 0.1, 36.8592796, -75.9844742]
[0.86, "2018-05-10", 0.29, 36.8592796, -75.9844742]
[2.17, "2017-09-28", 0, 36.8592796, -75.9844742]
[2.07, "2018-08-12", 0.117, 36.7883606, -76.0707779]
[1, "2016-07-16", 0.117, 36.7883606, -76.0707779]
[3.18, "2017-09-21", 0, 36.8272209, -76.1655579]
[2.16, "2018-12-02", 0.04, 36.8411407, -76.0571365]
[2.69, "2018-05-14", 0.03, 36.8247795, -75.9852524]
[1.3, "2019-03-29", 0, 36.8272209, -76.1655579]
[0.84, "2018-02-01", 0, 36.8433037, -76.1244736]
[1.14, "2016-05-18", 0.03, 36.8411407, -76

[1.35, "2016-05-01", 0.08, 36.8433037, -76.1244736]
[0.96, "2017-03-02", 0.03, 36.8272209, -76.1655579]
[0, "2019-02-27", 0.117, 36.8157768, -76.0609131]
[1.56, "2016-10-26", 0.117, 36.7116928, -76.268219]
[0.94, "2019-03-31", 0.13, 36.8433037, -76.1244736]
[1.34, "2018-06-30", 0, 36.8411407, -76.0571365]
[0, "2016-12-09", -0.41, 36.6798058, -75.984]
[3.07, "2017-08-30", 0.01, 36.8411407, -76.0571365]
[2.14, "2016-07-31", 0.117, 36.7883606, -76.0707779]
[1.83, "2017-07-19", 0.117, 36.7116928, -76.268219]
[0, "2018-06-15", 0.117, 36.8157768, -76.0609131]
[1.38, "2018-06-30", 0, 36.8433037, -76.1244736]
[2.32, "2018-11-10", 0, 36.8411407, -76.0571365]
[0, "2017-09-11", -0.77, 36.6798058, -75.984]
[0.13, "2017-10-14", -0.65, 36.6798058, -75.984]
[0.87, "2018-11-18", 0.117, 36.7883606, -76.0707779]
[1.54, "2016-07-09", 0, 36.8592796, -75.9844742]
[0.426509172, "2019-01-20", 0.117, 36.852, -76.2885666]
[1.32, "2018-02-18", 0.117, 36.7883606, -76.0707779]
[1.12, "2018-05-12", 0.117, 36.78836

...

[1.91, "2018-04-04", 0.11, 36.8247795, -75.9852524]
[1.52, "2016-10-08", 9.3, 36.8411407, -76.0571365]
[0.61, "2017-02-06", 0.117, 36.7883606, -76.0707779]
[0.04, "2018-12-02", 0.117, 36.8157768, -76.0609131]
[2.22, "2017-06-25", 0.117, 36.7116928, -76.268219]
[1.26, "2017-10-21", 0.117, 36.7883606, -76.0707779]
[0.55, "2017-02-18", 0, 36.8433037, -76.1244736]
[1.33, "2018-10-29", 0, 36.8433037, -76.1244736]
[1.24, "2016-10-30", 0, 36.8433037, -76.1244736]
[-14.698163, "2019-04-07", 0.117, 36.6927, -75.9263763]
[0.88, "2017-02-21", 0.117, 36.7883606, -76.0707779]
[-0.393700778, "2019-01-30", 0.117, 36.7211685, -76.0335922]
[2.15, "2017-06-24", 0.117, 36.7116928, -76.268219]
[3.19, "2018-09-08", 1.91, 36.8272209, -76.1655579]
[0.97, "2018-01-05", 0, 36.8411407, -76.0571365]
[2.43, "2018-05-17", 0.117, 36.7883606, -76.0707779]
[0.38, "2019-02-11", 0.54, 36.8411407, -76.0571365]
[1.18, "2017-10-05", 0, 36.8592796, -75.9844742]
[6.4, "2016-10-09", 3.82, 36.8272209, -76.1655579]
[1.85, "201

[1.24, "2018-04-01", 0, 36.8411407, -76.0571365]
[1.18, "2018-11-17", 0.117, 36.7883606, -76.0707779]
[1.45, "2019-02-07", 0, 36.8411407, -76.0571365]
[0.69, "2016-05-27", 0, 36.8592796, -75.9844742]
[1.2, "2017-07-07", 0.49, 36.6798058, -75.984]
[0.01, "2019-01-09", 0.13, 36.6798058, -75.984]
[1.29, "2017-06-15", 0, 36.8592796, -75.9844742]
[1.19, "2018-08-18", 0.117, 36.7883606, -76.0707779]
[1.65, "2019-01-18", 0, 36.8411407, -76.0571365]
[2.24, "2017-09-12", 0, 36.8247795, -75.9852524]
[1.59, "2018-12-16", 0.01, 36.8592796, -75.9844742]
[1.2, "2016-07-07", 0.79, 36.6798058, -75.984]
[0.36, "2016-11-24", -0.32, 36.6798058, -75.984]
[1.84, "2016-06-28", 0.117, 36.7116928, -76.268219]
[1.13, "2018-02-08", 0, 36.8433037, -76.1244736]
[1.48, "2016-08-27", 0.117, 36.7883606, -76.0707779]
[0, "2018-04-29", 0.3, 36.6798058, -75.984]
[0.37, "2017-02-05", 0, 36.8411407, -76.0571365]
[1.49, "2017-12-07", 0.117, 36.7883606, -76.0707779]
[1.56, "2018-10-15", 0, 36.8247795, -75.9852524]
[1.97, "

[1.69, "2018-04-25", 0.117, 36.7883606, -76.0707779]
[0.66, "2016-11-09", -0.58, 36.6798058, -75.984]
[1.64, "2019-03-23", 0.117, 36.7116928, -76.268219]
[1.7, "2017-08-03", 0.117, 36.7116928, -76.268219]
[0.1, "2016-11-14", -0.04, 36.6798058, -75.984]
[1.19, "2017-04-21", 0, 36.8433037, -76.1244736]
[2.69, "2018-06-13", 0.02, 36.8247795, -75.9852524]
[2.05, "2018-07-30", 1.17, 36.8272209, -76.1655579]
[0.52, "2018-02-21", 0.117, 36.7883606, -76.0707779]
[1.2, "2017-09-05", 0.117, 36.6798058, -75.984]
[1.56, "2017-02-02", 0, 36.8272209, -76.1655579]
[3.45, "2016-05-06", 0.117, 36.7116928, -76.268219]
[1.77, "2018-08-23", 0, 36.8433037, -76.1244736]
[1.2, "2019-01-26", 0.117, 36.8157768, -76.0609131]
[0.76, "2017-12-23", 0.117, 36.7883606, -76.0707779]
[0.7, "2017-04-12", 0, 36.8592796, -75.9844742]
[2.2, "2018-01-01", 0.117, 36.7116928, -76.268219]
[2.06, "2017-08-18", 0.22, 36.8247795, -75.9852524]
[0.18, "2018-12-01", 0.65, 36.6798058, -75.984]
[0.23, "2018-11-02", 0.117, 36.8157768,

[1.09, "2017-06-13", 0.117, 36.7116928, -76.268219]
[1.26, "2016-12-21", 0.117, 36.7116928, -76.268219]
[1.54, "2017-12-15", 0, 36.8247795, -75.9852524]
[0, "2018-06-08", 0.49, 36.6798058, -75.984]
[1.09, "2019-03-09", 0.01, 36.8592796, -75.9844742]
[2.31, "2018-12-18", 0.01, 36.8272209, -76.1655579]
[0.02, "2018-12-05", 0.16, 36.6798058, -75.984]
[2.02, "2018-10-08", 0, 36.8411407, -76.0571365]
[2.05, "2018-07-19", 0, 36.8247795, -75.9852524]
[1.38, "2016-09-14", 0.117, 36.7883606, -76.0707779]
[2.75, "2017-05-30", 0.117, 36.7116928, -76.268219]
[1.46, "2018-02-24", 0, 36.8272209, -76.1655579]
[1.77, "2016-06-16", 0.02, 36.8411407, -76.0571365]
[2.96, "2018-04-16", 0.81, 36.8272209, -76.1655579]
[0, "2018-03-22", 0.117, 36.8157768, -76.0609131]
[1.85, "2019-03-22", 0.01, 36.8411407, -76.0571365]
[0.95, "2016-12-16", 0.117, 36.7883606, -76.0707779]
[0.35, "2018-11-05", 0.117, 36.6798058, -75.984]
[1.89, "2017-10-09", 0, 36.8272209, -76.1655579]
[2.41, "2018-10-13", 0, 36.8592796, -75.9

[2.02, "2018-03-09", 0.117, 36.7883606, -76.0707779]
[1.68, "2017-09-07", 0, 36.8433037, -76.1244736]
[0.27, "2019-01-23", 0, 36.8592796, -75.9844742]
[2.4, "2016-11-12", 0, 36.8247795, -75.9852524]
[1.77, "2016-12-09", 0, 36.8411407, -76.0571365]
[1.6, "2018-01-02", 0, 36.8433037, -76.1244736]
[2.19, "2017-04-06", 0.16, 36.8272209, -76.1655579]
[2.03, "2018-06-13", 0.01, 36.8433037, -76.1244736]
[1.21, "2016-05-27", 0.117, 36.7116928, -76.268219]
[1.58, "2018-05-01", 0, 36.8272209, -76.1655579]
[2.7, "2016-12-15", 0.117, 36.7116928, -76.268219]
[1.44, "2017-10-11", 0.29, 36.8592796, -75.9844742]
[1.88, "2017-10-19", 0, 36.8247795, -75.9852524]
[1.78, "2016-08-29", 0, 36.8411407, -76.0571365]
[1.36, "2017-11-29", 0, 36.8247795, -75.9852524]
[0, "2018-08-05", 0.88, 36.6798058, -75.984]
[1.81, "2019-03-12", 0, 36.8272209, -76.1655579]
[1.86, "2018-02-26", 0.03, 36.8272209, -76.1655579]
[1.59, "2017-06-09", 0.117, 36.7883606, -76.0707779]
[2.3, "2017-09-16", 0.117, 36.7116928, -76.268219]

In [124]:
def my_gen(i):
    yield ([i[3], i[2], i[5], i[6], i[7]])

In [132]:
my_gen(train_ds).get_next()


AttributeError: 'generator' object has no attribute 'get_next'

TypeError: 'TakeDataset' object is not subscriptable

In [137]:
# Transforms a scalar string `example_proto` into a pair of a scalar string and
# a scalar integer, representing an image and its label, respectively.
def _parse_function(a,b,c,d,e,f,g,h):
    # share, dtype, default value
    features = {"image": tf.io.FixedLenFeature((), tf.string, default_value=""),
                "label": tf.io.FixedLenFeature((), tf.int64, default_value=0)}


    parsed_features = tf.io.parse_single_example(a, features)

    return parsed_features["image"], parsed_features["label"]

# Creates a dataset that reads all of the examples from two files, and extracts
# the image and label features.
#filenames = ["/var/data/file1.tfrecord", "/var/data/file2.tfrecord"]
#dataset = tf.data.TFRecordDataset(filenames)
foo_dataset = train_ds.map(_parse_function)

In [141]:
my_dataset_head(foo_dataset)

InvalidArgumentError: Could not parse example input, value: 'USGS'
	 [[{{node ParseSingleExample/ParseSingleExample}}]] [Op:IteratorGetNextSync]

In [145]:
foo = tf.keras.backend.squeeze(train_ds,1)

ValueError: Attempt to convert a value (<TakeDataset shapes: ((), (), (), (), (), (), (), ()), types: (tf.string, tf.int64, tf.string, tf.float32, tf.float32, tf.float32, tf.float32, tf.float32)>) with an unsupported type (<class 'tensorflow.python.data.ops.dataset_ops.TakeDataset'>) to a Tensor.

In [146]:
tf.keras.utils.serialize_keras_object(train_ds)

ValueError: ('Cannot serialize', <TakeDataset shapes: ((), (), (), (), (), (), (), ()), types: (tf.string, tf.int64, tf.string, tf.float32, tf.float32, tf.float32, tf.float32, tf.float32)>)

In [147]:
tf.is_tensor(train_ds)

False

In [153]:
# Iterate over the batches of a dataset.
#for x_batch_train, y_batch_train in train_ds:
#    with tf.GradientTape() as tape:
#        logits = layer(x_batch_train)  # Logits for this minibatch
#        # Loss value for this minibatch
#        loss_value = loss_fn(y_batch_train, logits)
#        # Add extra losses created during this forward pass:
#        loss_value += sum(model.losses)

#        grads = tape.gradient(loss_value, model.trainable_variables)
#        optimizer.apply_gradients(zip(grads, model.trainable_variables))

In [165]:

#tf.print([i[3], i[2], i[5], i[6], i[7]])


model = tf.keras.Sequential([
    # Adds a densely-connected layer with 64 units to the model:
    layers.Dense(64, activation='relu', input_shape=(8,)),
    # Add another:
    layers.Dense(64, activation='relu'),
    # Add a softmax layer with 10 output units:
    layers.Dense(10, activation='softmax')])

In [166]:
# this was for the previous
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [167]:
# Configure a model for mean-squared error regression.
model.compile(optimizer=tf.keras.optimizers.Adam(0.01),
              loss='mse',       # mean squared error
              metrics=['mae'])  # mean absolute error

In [168]:
# using this page for training with datasets
# https://www.tensorflow.org/alpha/guide/keras/overview

In [169]:
# Instantiates a toy dataset instance:
# WTF is data?
#dataset = tf.data.Dataset.from_tensor_slices((data, labels))
#dataset = dataset.batch(32)



dataset = train_ds.batch(32)  # this will take five rows

# Don't forget to specify `steps_per_epoch` when calling `fit` on a dataset.
model.fit(dataset, epochs=10, steps_per_epoch=30)

Epoch 1/10


ValueError: Output of generator should be a tuple `(x, y, sample_weight)` or `(x, y)` or (x,). Found: (<tf.Tensor: id=1397687, shape=(32,), dtype=string, numpy=
array([b'USGS', b'GSVB', b'USGS', b'USGS', b'USGS', b'USGS', b'USGS',
       b'USGS', b'USGS', b'USGS', b'USGS', b'USGS', b'USGS', b'GSVB',
       b'USGS', b'GSVB', b'USGS', b'USGS', b'GSVB', b'GSVB', b'USGS',
       b'USGS', b'USGS', b'GSVB', b'USGS', b'USGS', b'USGS', b'USGS',
       b'USGS', b'USGS', b'USGS', b'USGS'], dtype=object)>, <tf.Tensor: id=1397688, shape=(32,), dtype=int64, numpy=
array([  2043155,  33670002, 204288539, 204291317, 204291317,   2043155,
       204291317, 204291317, 204288539, 204295505, 204288539, 204297575,
       204288721,  33670009, 204288539,  33670003, 204300267, 204297575,
        33670003,  33670007, 204293125, 204300267, 204297575,  33670006,
       204291317, 204288721,   2043155, 204297575, 204288539, 204291317,
         2042928, 204288721])>, <tf.Tensor: id=1397689, shape=(32,), dtype=st

In [178]:
### A regressor
model = keras.Sequential([
    layers.Dense(64, activation=tf.nn.relu, input_shape=[5]),
    layers.Dense(64, activation=tf.nn.relu),
    layers.Dense(1)
  ])



In [179]:
optimizer = tf.keras.optimizers.RMSprop(0.001)

model.compile(loss='mean_squared_error',
                optimizer=optimizer,
                metrics=['mean_absolute_error', 'mean_squared_error'])

In [199]:
#iter = train_ds.make_one_shot_iterator()
another_ds = tf.Variable("Elephant", tf.string)
for i in train_ds:
    inputs = i[0]
    another_ds = tf.concat(another_ds, inputs)

ValueError: Tensor conversion requested dtype int32 for Tensor with dtype string: "tf.Tensor(b'USGS', shape=(), dtype=string)"

In [200]:
inputs

<tf.Tensor: id=1453915, shape=(), dtype=string, numpy=b'USGS'>

In [201]:
another_ds

<tf.Variable 'Variable:0' shape=() dtype=string, numpy=b'Elephant'>